In [1]:
def analyze_orders(orders):
    filtered_orders = []

    for order in orders:
        customer = order["customer"]
        has_digit = False
        for char in customer:
            if char.isdigit():
                has_digit = True
                break

        if not has_digit:
            new_customer = customer[0].upper() + customer[1:].lower()
            filtered_orders.append({
                "order_id": order["order_id"],
                "customer": new_customer,
                "items": order["items"][:],
                "notes": order["notes"][:]
            })

    all_vowels_set = set()
    unique_products_set = set()
    orders_with_processed = []

    for order in filtered_orders:
        processed_items = []

        for item in order["items"]:
            name = item["name"]
            price = item["price"]
            quantity = item["quantity"]

            if price <= 0:
                continue

            if price > 100 and quantity > 1:
                price = price * quantity

            if quantity % 2 == 1:
                price_int = int(price)
                sum_digits = 0
                temp = price_int
                while temp > 0:
                    sum_digits += temp % 10
                    temp //= 10
                price += sum_digits

            processed_items.append({
                "name": name,
                "price": round(price, 2),
                "quantity": quantity
            })

            unique_products_set.add(name)

        orders_with_processed.append({
            "order_id": order["order_id"],
            "customer": order["customer"],
            "processed_items": processed_items,
            "notes": order["notes"]
        })

    word_order_count = {}
    punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"

    for order in orders_with_processed:
        all_notes = " ".join(order["notes"])

        clean_notes = ""
        for char in all_notes:
            if char not in punctuation:
                clean_notes += char

        words = clean_notes.split()

        unique_words = []
        seen_words = set()

        for word in words:
            word_lower = word.lower()
            if len(word_lower) >= 4 and word_lower != word_lower[::-1]:
                if word_lower not in seen_words:
                    seen_words.add(word_lower)
                    unique_words.append(word_lower)

        vowels = set("aeiouy")
        order_vowels = set()

        for word in unique_words:
            for char in word:
                if char in vowels:
                    order_vowels.add(char)

        all_vowels_set.update(order_vowels)

        for word in unique_words:
            if word not in word_order_count:
                word_order_count[word] = set()
            word_order_count[word].add(order["order_id"])

    word_final = {}
    for word, orders_set in word_order_count.items():
        if len(orders_set) >= 2:
            word_final[word] = len(orders_set)

    word_items = list(word_final.items())
    for i in range(len(word_items)):
        for j in range(i + 1, len(word_items)):
            if word_items[i][1] < word_items[j][1]:
                word_items[i], word_items[j] = word_items[j], word_items[i]
            elif word_items[i][1] == word_items[j][1] and word_items[i][0] > word_items[j][0]:
                word_items[i], word_items[j] = word_items[j], word_items[i]

    word_counts = dict(word_items)

    orders_with_total = []
    for order in orders_with_processed:
        total = 0
        for item in order["processed_items"]:
            total += item["price"] * item["quantity"]
        orders_with_total.append((order["order_id"], total))

    for i in range(len(orders_with_total)):
        for j in range(i + 1, len(orders_with_total)):
            if orders_with_total[i][1] < orders_with_total[j][1]:
                orders_with_total[i], orders_with_total[j] = orders_with_total[j], orders_with_total[i]
            elif orders_with_total[i][1] == orders_with_total[j][1] and orders_with_total[i][0] > orders_with_total[j][0]:
                orders_with_total[i], orders_with_total[j] = orders_with_total[j], orders_with_total[i]

    orders_by_total = [order_id for order_id, _ in orders_with_total]

    orders_by_item_count = {}
    seen_orders = set()

    for order in orders_with_processed:
        order_id = order["order_id"]
        if order_id not in seen_orders:
            seen_orders.add(order_id)
            item_count = len(order["processed_items"])
            if item_count not in orders_by_item_count:
                orders_by_item_count[item_count] = []
            orders_by_item_count[item_count].append(order_id)

    result = {
        "orders": [
            {
                "order_id": o["order_id"],
                "customer": o["customer"],
                "processed_items": o["processed_items"]
            }
            for o in orders_with_processed
        ],
        "word_counts": word_counts,
        "all_vowels": all_vowels_set,
        "unique_products": unique_products_set,
        "orders_by_total": orders_by_total,
        "orders_by_item_count": orders_by_item_count
    }

    return result


orders = [
    {
        "order_id": "A123",
        "customer": "john_doe42",
        "items": [
            {"name": "Laptop", "price": 999.99, "quantity": 1},
            {"name": "Mouse2", "price": 25, "quantity": 2}
        ],
        "notes": ["Deliver ASAP", "fragile package", "handle with care"]
    },
    {
        "order_id": "B456",
        "customer": "alice_smith",
        "items": [
            {"name": "Keyboard", "price": 75.50, "quantity": 3},
            {"name": "Monitor", "price": 299.99, "quantity": 1},
            {"name": "USB Cable", "price": 15, "quantity": 5}
        ],
        "notes": ["Express delivery", "gift wrap", "fragile"]
    },
    {
        "order_id": "C789",
        "customer": "bob_jones123",
        "items": [
            {"name": "Tablet", "price": 399.99, "quantity": 2},
            {"name": "Case", "price": 29.99, "quantity": 2}
        ],
        "notes": ["weekend delivery", "fragile items"]
    },
    {
        "order_id": "D012",
        "customer": "charlie_brown",
        "items": [
            {"name": "Book", "price": 19.99, "quantity": 4},
            {"name": "Pen Set", "price": 12.50, "quantity": 2},
            {"name": "Notebook", "price": 8.75, "quantity": 3}
        ],
        "notes": ["school supplies", "urgent", "handle with care"]
    }
]

print("Результат анализа заказов:")
result = analyze_orders(orders)

print("\n1. Обработанные заказы:")
for order in result["orders"]:
    print(f"  {order['order_id']} - {order['customer']}")
    for item in order["processed_items"]:
        print(f"    {item['name']}: {item['price']} x {item['quantity']}")

print("\n2. Словарь слов (встречаются в >=2 заказах):")
for word, count in result["word_counts"].items():
    print(f"  {word}: {count}")

print("\n3. Все гласные из заметок:")
print(f"  {result['all_vowels']}")

print("\n4. Уникальные товары:")
print(f"  {result['unique_products']}")

print("\n5. Заказы по общей сумме:")
for order_id in result["orders_by_total"]:
    print(f"  {order_id}")

print("\n6. Заказы по количеству товаров:")
for count, order_ids in result["orders_by_item_count"].items():
    print(f"  {count} товар(ов): {order_ids}")

Результат анализа заказов:

1. Обработанные заказы:
  B456 - Alice_smith
    Keyboard: 87.5 x 3
    Monitor: 319.99 x 1
    USB Cable: 21 x 5
  D012 - Charlie_brown
    Book: 19.99 x 4
    Pen Set: 12.5 x 2
    Notebook: 16.75 x 3

2. Словарь слов (встречаются в >=2 заказах):

3. Все гласные из заметок:
  {'o', 'y', 'a', 'i', 'u', 'e'}

4. Уникальные товары:
  {'Notebook', 'Pen Set', 'Monitor', 'Book', 'Keyboard', 'USB Cable'}

5. Заказы по общей сумме:
  B456
  D012

6. Заказы по количеству товаров:
  3 товар(ов): ['B456', 'D012']
